In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark  = SparkSession.builder.appName("IPL_WORK").getOrCreate()

In [3]:
venue = spark.read.format("csv").options(header="true",inferSchema="true").load("ipl_venue.csv")
matches = spark.read.format("csv").options(header="true",inferSchema="true").load("ipl_matches.csv")
ball_by_ball = spark.read.format("csv").options(header="true",inferSchema="true").load("ipl_ball_by_ball.csv")

In [4]:
venue.count()
matches.count()
ball_by_ball.count()

193468

In [5]:
venue.show()

+--------+--------------------+--------------+
|venue_id|               venue|          city|
+--------+--------------------+--------------+
|       1|Dr. Y.S. Rajasekh...| Visakhapatnam|
|       2|Sharjah Cricket S...|       Sharjah|
|       3|JSCA Internationa...|        Ranchi|
|       4|Saurashtra Cricke...|        Rajkot|
|       5|Shaheed Veer Nara...|        Raipur|
|       6|Maharashtra Crick...|          Pune|
|       7|Subrata Roy Sahar...|          Pune|
|       8|    St George's Park|Port Elizabeth|
|       9|Vidarbha Cricket ...|        Nagpur|
|      12|   Brabourne Stadium|        Mumbai|
|      13|Dr DY Patil Sport...|        Mumbai|
|      14|    Wankhede Stadium|        Mumbai|
|      15|        Eden Gardens|       Kolkata|
|      16|       Nehru Stadium|         Kochi|
|      17|De Beers Diamond ...|     Kimberley|
|      18|          Green Park|        Kanpur|
|      19|New Wanderers Sta...|  Johannesburg|
|      20|Sawai Mansingh St...|        Jaipur|
|      21|Hol

In [6]:
matches.show(100)

+--------+----------+----------------+--------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+---------------+---------------+
|match_id|      date| player_of_match|venue_id|neutral_venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|eliminator|method|        umpire1|        umpire2|
+--------+----------+----------------+--------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+---------------+---------------+
|  335982|2008-04-18|     BB McCullum|      35|            0|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|         N|    NA|      Asad Rauf|    RE Koertzen|
|  335983|2008-04-19|      MEK Hussey|      31|            0|     Kings 

In [7]:
ball_by_ball.show()

+--------+------+-----+----+-------------+-------------+------------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+
|match_id|inning|overs|ball|      batsman|  non_striker|      bowler|batsman_runs|extra_runs|total_runs|non_boundary|is_wicket|dismissal_kind|player_dismissed|fielder|extras_type|        batting_team|        bowling_team|
+--------+------+-----+----+-------------+-------------+------------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+
|  419157|     2|   16|   4|      V Kohli|R Vinay Kumar|CRD Fernando|           0|         0|         0|           0|        0|            NA|              NA|     NA|         NA|Royal Challengers...|      Mumbai Indians|
|  419157|     2|   16|   5|      V Kohli|R Vinay Kumar|CRD Fernando|           1|         0|         1|        

In [8]:
# Find the top 3 venues which hosted the most number of eliminator matches?
print(venue.columns)
print(matches.columns)


['venue_id', 'venue', 'city']
['match_id', 'date', 'player_of_match', 'venue_id', 'neutral_venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2']


In [9]:
top3 = venue.join(matches,"venue_id","inner")
top3.filter(col("eliminator")=="Y").groupBy("venue").agg(count("eliminator")).orderBy(col("count(eliminator)").desc()).limit(3).show(3,False)

+-----------------------------------+-----------------+
|venue                              |count(eliminator)|
+-----------------------------------+-----------------+
|Dubai International Cricket Stadium|3                |
|Sheikh Zayed Stadium               |2                |
|Sardar Patel Stadium, Motera       |1                |
+-----------------------------------+-----------------+



In [10]:
#Return most number of catches taken by a player in IPL history?
ball_by_ball.select("dismissal_kind").distinct().show()

df = ball_by_ball.groupBy("fielder","dismissal_kind").agg(count("dismissal_kind").alias("Total_dismissal_kind"))
df.filter(col("dismissal_kind")=="caught").orderBy(col("Total_dismissal_kind").desc()).limit(1).show()


+--------------------+
|      dismissal_kind|
+--------------------+
|             stumped|
|          hit wicket|
|                  NA|
|              bowled|
|                 lbw|
|   caught and bowled|
|        retired hurt|
|              caught|
|             run out|
|obstructing the f...|
+--------------------+

+----------+--------------+--------------------+
|   fielder|dismissal_kind|Total_dismissal_kind|
+----------+--------------+--------------------+
|KD Karthik|        caught|                 118|
+----------+--------------+--------------------+



In [11]:
#Write a query to return a report for highest wicket taker in matches which were affected by Duckworth-Lewis’s method (D/L method).

high = matches.join(ball_by_ball,"match_id","inner").filter(col("method")=="D/L")
high.groupBy("bowler").agg(sum("is_wicket").alias("Total_Wicket_in_D/L")).orderBy(col("Total_Wicket_in_D/L").desc()).limit(1).show()


+--------+-------------------+
|  bowler|Total_Wicket_in_D/L|
+--------+-------------------+
|AB Dinda|                  8|
+--------+-------------------+



In [12]:
# Write a query to return a report for highest strike rate by a batsman in non powerplay overs(7-20 overs)
highest_runrate = ball_by_ball.filter((col("overs")>7) & (col("overs")<20))\
                            .groupBy("batsman").agg(sum("batsman_runs").alias("Total_runs_score"),count("ball").alias("Total_balls_played"))\
                            .withColumn("Strike_rate",round(col("Total_runs_score")/col("Total_balls_played")*100,2))\
                            .orderBy(col("Strike_rate").desc()).limit(1)


highest_runrate.show()

#Strike Rate = (Runs Scored / Balls Faced) x 100 

+----------+----------------+------------------+-----------+
|   batsman|Total_runs_score|Total_balls_played|Strike_rate|
+----------+----------------+------------------+-----------+
|B Stanlake|               5|                 2|      250.0|
+----------+----------------+------------------+-----------+



In [13]:
#Write a query to return a report for highest extra runs in a venue (stadium, city).

high = venue.join(matches,"venue_id","inner").join(ball_by_ball,"match_id","inner")

high.groupBy("venue_id","venue").agg(sum("extra_runs").alias("Total_Extra_Runs")).orderBy(col("Total_Extra_Runs").desc()).limit(1).show(truncate=False)

+--------+---------------------+----------------+
|venue_id|venue                |Total_Extra_Runs|
+--------+---------------------+----------------+
|35      |M.Chinnaswamy Stadium|1355            |
+--------+---------------------+----------------+



In [14]:
#Write a query to return a report for the cricketers with the most number of players of the match award in neutral venues.
matches.filter(col("neutral_venue")=="1").groupBy("player_of_match","neutral_venue").agg(count("match_id").alias("Total_Number_POM"))\
                                                                                         .orderBy(col("Total_Number_POM").desc()).show()


+----------------+-------------+----------------+
| player_of_match|neutral_venue|Total_Number_POM|
+----------------+-------------+----------------+
|       JH Kallis|            1|               3|
|       JP Duminy|            1|               3|
|       YK Pathan|            1|               3|
|      GJ Maxwell|            1|               3|
|  M Muralitharan|            1|               2|
|   KC Sangakkara|            1|               2|
|     LRPL Taylor|            1|               2|
|    Yuvraj Singh|            1|               2|
|  Sandeep Sharma|            1|               2|
|    SR Tendulkar|            1|               2|
|        SK Raina|            1|               2|
|       RG Sharma|            1|               2|
|    AC Gilchrist|            1|               2|
|DPMD Jayawardene|            1|               2|
|  AB de Villiers|            1|               2|
|       ML Hayden|            1|               2|
|        DR Smith|            1|               2|


In [15]:
#Write a query to find out who has officiated (as an umpire) the most number of matches in IPL.

matches.groupBy("umpire1").agg(count("match_id").alias("Total_Matches")).orderBy(col("Total_Matches").desc()).limit(1).show()
matches.groupBy("umpire2").agg(count("match_id").alias("Total_Matches")).orderBy(col("Total_Matches").desc()).limit(1).show()

matches.groupBy("umpire1","umpire2").agg(count("match_id").alias("Total_Matches")).orderBy(col("Total_Matches").desc()).limit(1).show()


+---------------+-------------+
|        umpire1|Total_Matches|
+---------------+-------------+
|HDPK Dharmasena|           78|
+---------------+-------------+

+-------+-------------+
|umpire2|Total_Matches|
+-------+-------------+
| S Ravi|           84|
+-------+-------------+

+-------+---------+-------------+
|umpire1|  umpire2|Total_Matches|
+-------+---------+-------------+
| S Ravi|RJ Tucker|           12|
+-------+---------+-------------+



In [16]:
#Find venue details of the match where V Kohli scored his highest individual runs in IPL.
viru = venue.join(matches,"venue_id","inner").join(ball_by_ball,"match_id","inner")

viru.groupBy("venue","batsman","match_id").agg(sum("batsman_runs").alias("Total_Individual_runs")).filter(col("batsman")=="V Kohli")\
.orderBy(col("Total_Individual_runs").desc()).limit(1).show()

+--------------------+-------+--------+---------------------+
|               venue|batsman|match_id|Total_Individual_runs|
+--------------------+-------+--------+---------------------+
|M.Chinnaswamy Sta...|V Kohli|  980999|                  113|
+--------------------+-------+--------+---------------------+



In [17]:
#How winning/ loosing tosses can impact a match and its result
print(matches.columns)

####### WRONG  ######
df = matches.withColumn("winner_binary_team1",when(col("toss_winner")==col("team1"),1).otherwise(0))\
            .withColumn("winner_binary_team2",when(col("winner_binary_team1")==0,1))

df.groupBy("team1","team2","toss_winner").agg(count("winner_binary_team1"),count("winner_binary_team2")).show()

#######    RIGHT  #############

df2 = matches.withColumn("result_win_loss",when(col("toss_winner")==col("winner"),"Win").otherwise("Lost"))
df2.groupBy("team1","result_win_loss").agg(count("result_win_loss").alias("Count_Result")).orderBy("team1").show(100)


['match_id', 'date', 'player_of_match', 'venue_id', 'neutral_venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2']
+--------------------+--------------------+--------------------+--------------------------+--------------------------+
|               team1|               team2|         toss_winner|count(winner_binary_team1)|count(winner_binary_team2)|
+--------------------+--------------------+--------------------+--------------------------+--------------------------+
|     Deccan Chargers| Chennai Super Kings|     Deccan Chargers|                         2|                         0|
|     Deccan Chargers|     Kings XI Punjab|     Kings XI Punjab|                         4|                         4|
|      Mumbai Indians|    Rajasthan Royals|      Mumbai Indians|                         6|                         0|
|    Rajasthan Royals|      Delhi Capitals|      Delhi Capitals|                    

In [18]:
#Write a query to get a list of top 10 players with the highest batting average Note:
#Batting average is the total number of runs scored divided by the number of times they have been out 
#(Make sure to include run outs (on non-striker end) as valid out while calculating average).


ball_by_ball.groupBy("batsman").agg(round(sum("batsman_runs")).alias("Total_run"),sum("is_wicket").alias("Total_wicket")).orderBy(col("Total_run").desc())\
                .withColumn("Average_runs",round(col("Total_run")/col("Total_wicket"))).show()


df = ball_by_ball.filter(col("dismissal_kind")!="NA")\
                        .groupBy("batsman").agg(round(sum("batsman_runs")).alias("Total_run"),\
                           count("dismissal_kind").alias("Total_Out"))


df.withColumn("Batting_Average",round(col("Total_run")/col("Total_Out"),2)).orderBy(col("Total_run").desc()).limit(10).show()


+--------------+---------+------------+------------+
|       batsman|Total_run|Total_wicket|Average_runs|
+--------------+---------+------------+------------+
|       V Kohli|     5878|         163|        36.0|
|      SK Raina|     5368|         160|        34.0|
|     DA Warner|     5254|         126|        42.0|
|     RG Sharma|     5230|         177|        30.0|
|      S Dhawan|     5197|         152|        34.0|
|AB de Villiers|     4849|         114|        43.0|
|      CH Gayle|     4772|         116|        41.0|
|      MS Dhoni|     4632|         126|        37.0|
|    RV Uthappa|     4607|         169|        27.0|
|     G Gambhir|     4217|         135|        31.0|
|     AM Rahane|     3933|         125|        31.0|
|     SR Watson|     3874|         125|        31.0|
|    KD Karthik|     3823|         152|        25.0|
|     AT Rayudu|     3659|         123|        30.0|
|     MK Pandey|     3268|         109|        30.0|
|     YK Pathan|     3204|         111|       

In [19]:
matches.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue_id: integer (nullable = true)
 |-- neutral_venue: integer (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)



In [20]:
venue.printSchema()

root
 |-- venue_id: integer (nullable = true)
 |-- venue: string (nullable = true)
 |-- city: string (nullable = true)



In [21]:
ball_by_ball.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- inning: integer (nullable = true)
 |-- overs: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- batsman_runs: integer (nullable = true)
 |-- extra_runs: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- non_boundary: integer (nullable = true)
 |-- is_wicket: integer (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- fielder: string (nullable = true)
 |-- extras_type: string (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)



In [22]:
ball_by_ball.show(100)

+--------+------+-----+----+-------------+-------------+---------------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+
|match_id|inning|overs|ball|      batsman|  non_striker|         bowler|batsman_runs|extra_runs|total_runs|non_boundary|is_wicket|dismissal_kind|player_dismissed|fielder|extras_type|        batting_team|        bowling_team|
+--------+------+-----+----+-------------+-------------+---------------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+
|  419157|     2|   16|   4|      V Kohli|R Vinay Kumar|   CRD Fernando|           0|         0|         0|           0|        0|            NA|              NA|     NA|         NA|Royal Challengers...|      Mumbai Indians|
|  419157|     2|   16|   5|      V Kohli|R Vinay Kumar|   CRD Fernando|           1|         0|    